In [ ]:
# скачивание и распаковка проекта, распаковка данных
!wget https://github.com/gimaevra94/gan/archive/refs/heads/main.zip
!unzip /content/main.zip
!unzip /content/gan-main/data.zip

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from tqdm import tqdm_notebook as tqdm

class DCGAN(tf.keras.Model):

    def __init__(self):
        super().__init__()

        # лосс, инициализатор весов, оптимайзер, генератор, дискриминатор
        self.loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
        self.init=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
        self.optimizer = tf.keras.optimizers.legacy.Adam(0.0002, beta_1=0.5, beta_2=0.999)
        self.generator = self._build_generator()
        self.discriminator = self._build_discriminator()

    def discriminator_loss(self, real_output, fake_output):
        """считает ошибку между выходами дискриминатора и метками классов 1,0"""
        real_loss = self.loss(tf.ones_like(real_output), real_output)
        fake_loss = self.loss(tf.zeros_like(fake_output), fake_output)
        return real_loss + fake_loss

    def generator_loss(self, fake_output):
        """считает ошибку между выходом дискриминатора и метками класса 1"""
        return self.loss(tf.ones_like(fake_output), fake_output)

    def _build_generator(self):
        """принимает вектор из нормального распределения
        и  постепенно уменьшая количество каналов
        и увеличивая размер изображения превращает его в 80,80,1"""
        model_gen = tf.keras.Sequential([
            tf.keras.layers.Reshape(target_shape=(1, 1, 200), input_shape=(200,)),

            tf.keras.layers.UpSampling2D(4),
            tf.keras.layers.Conv2D(256,
                            kernel_size=4,
                            padding="same",
                            use_bias=False,
                            kernel_initializer=self.init),

            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),

            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(128,
                            kernel_size=4,
                            padding="same",
                            use_bias=False,
                            kernel_initializer=self.init),

            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),

            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(64,
                            kernel_size=4,
                            padding="same",
                            use_bias=False,
                            kernel_initializer=self.init),

            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),

            tf.keras.layers.UpSampling2D(),
            tf.keras.layers.Conv2D(1,
                            kernel_size=4,
                            strides=1,
                            activation="tanh",
                            padding="same",
                            kernel_initializer=self.init)])
        return model_gen

    def _build_discriminator(self):
        """принимает на вход реальное изображение
        либо сгенерированное. Постепенно уменьшает его до вектора логитов"""
        model_disc = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=((32,32,1))),

            tf.keras.layers.Conv2D(64,
                              kernel_size=4,
                              padding="same",
                              use_bias=False,
                              kernel_initializer=self.init,
                              strides=2),

            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),

            tf.keras.layers.Conv2D(128,
                                kernel_size=4,
                                padding="same",
                                use_bias=False,
                                kernel_initializer=self.init,
                                strides=2),

            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),

            tf.keras.layers.Conv2D(256,
                                kernel_size=4,
                                padding="same",
                                use_bias=False,
                                kernel_initializer=self.init,
                                strides=2),

            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.LeakyReLU(),

            tf.keras.layers.Conv2D(1,kernel_size=4,kernel_initializer=self.init),

            tf.keras.layers.Flatten()])

        return model_disc

    @tf.function
    def train_step(self, images):
        """1. генератор принимает вектор нормального распределения и выдает фейк.данные
           2. дискриминатор принимает фейк.данные генератора и реальные данные
           3. выход дискриминатора на реальных и фейк.данных едет в discriminator_loss
           4. выход дискриминатора на фейк.данных едет в generator_loss
           5. с помощью лосса генератора и обученных параметров генератора
           считается градиент для генератора. аналогично для дискриминатора
           6. оба градиента пробрасываются в оптимайзер"""
        noise = tf.random.normal([tf.cast(images.shape[0], tf.int32),200])

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = self.generator(noise, training=True)

            real_output = self.discriminator(images, training=True)
            fake_output = self.discriminator(generated_images, training=True)

            gen_loss = self.generator_loss(fake_output)
            disc_loss = self.discriminator_loss(real_output, fake_output)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))

    def train(self, dataset):
      """каждую итерацию один батч реальных данных едет в метод train_step,
      откуда поедет на вход дискриминатора"""
        for epoch in range(5):
            for image_batch in tqdm(dataset):
                self.train_step(image_batch)

gan = DCGAN()

#path='C:/Users/gimaevra94/Documents/local/gan/data'
path='/content'

data=pathlib.Path(path).parent/'data'
data2=data/'data2'
data3=data2/'data3'

train=tf.keras.utils.image_dataset_from_directory(directory=data2,
                                                  label_mode=None,
                                                  batch_size=256,
                                                  color_mode='grayscale',
                                                  image_size=(80,80))

norm  = tf.keras.layers.Rescaling(1./127.5, offset=-1)
train = train.map(lambda x: norm(x))

gan.train(train)